In [10]:
!pip install jupyter notebook
!pip install jupyter_http_over_ws
!jupyter serverextension enable --py jupyter_http_over_ws



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for jupyter_http_over_ws from https://files.pythonhosted.org/packages/71/b8/d0047d068facc913a354d14c4664b0ba99b8f30f8d66a1800e98549c06ce/jupyter_http_over_ws-0.0.8-py2.py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot tru

SyntaxError: invalid syntax (2742849633.py, line 1)

In [6]:
import numpy as np
import tensorflow as tf
import glob
import rasterio
import geopandas as gpd
from rasterio.features import geometry_mask
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Define the directories containing images and shapefiles
image_dir = "C:\\Users\\Student\\Documents\\101_walrus_data\\train\\clipped_images"
shapefile_dir = "C:\\Users\\Student\\Documents\\101_walrus_data\\train\\clipped_train"

# Use glob to list all .tif and .shp files in the specified directories
image_paths = glob.glob(f"{image_dir}/*.tif")
shapefile_paths = glob.glob(f"{shapefile_dir}/*.shp")

# Function to extract island name and date from file names
def extract_island_and_date(filename):
    pattern = r'([a-zA-Z]+)_(\d{8})'
    match = re.search(pattern, filename)
    if match:
        return match.group(1), match.group(2)
    return None, None

# Pair image and shapefile paths based on filenames
def pair_files(image_paths, shapefile_paths):
    file_pairs = {}
    for image_path in image_paths:
        island, date = extract_island_and_date(os.path.basename(image_path))
        if island is None or date is None:
            print(f"Failed to extract island and date from {image_path}")
            continue

        matching_shapefiles = [shapefile for shapefile in shapefile_paths if island in shapefile and date in shapefile]
        if not matching_shapefiles:
            print(f"No matching shapefile found for {image_path}")
            continue

        shapefile_path = matching_shapefiles[0]
        file_pairs[image_path] = shapefile_path
    return file_pairs

file_pairs = pair_files(image_paths, shapefile_paths)

# Function to create mask from shapefiles
def create_mask(image_path, shapefile_path):
    with rasterio.open(image_path) as src:
        image = src.read()  # Read all bands
        transform = src.transform

    gdf = gpd.read_file(shapefile_path)
    mask = np.zeros((src.height, src.width), dtype=np.uint8)

    # Apply label mapping
    label_mapping = {
        'ocean': 1,
        'rock': 2,
        'freshwater': 3,
        'ice': 4,
        'walrus_pure': 5,
        'walrus_mixed': 6
    }

    for _, row in gdf.iterrows():
        geom_mask = geometry_mask([row['geometry']], transform=transform, invert=True, out_shape=(src.height, src.width))
        label = label_mapping.get(row['label'], 0)  # Use 0 for any label not in the mapping
        mask[geom_mask] = label

    return image, mask

No matching shapefile found for C:\Users\Student\Documents\101_walrus_data\train\clipped_images\pool_20230803_image.tif


In [8]:
# Function to ensure masks have valid labels
def validate_and_correct_masks(masks, num_classes):
    unique_values = np.unique(masks)
    print("Unique values in masks before correction:", unique_values)
    
    # Ensure all values are within the valid range
    masks[masks >= num_classes] = 0

    unique_values = np.unique(masks)
    print("Unique values in masks after correction:", unique_values)
    return masks

# Create dataset of image-mask pairs
def get_image_mask_pairs(file_pairs):
    images = []
    masks = []

    for image_path, shapefile_path in file_pairs.items():
        image, mask = create_mask(image_path, shapefile_path)
        images.append(image)
        masks.append(mask)

    images = np.stack(images)
    masks = np.stack(masks)
    
    # Validate and correct masks
    masks = validate_and_correct_masks(masks, num_classes=6)
    
    return images, masks

# Function to pad images and masks to a fixed size
def pad_to_shape(image, mask, target_shape=(256, 256)):
    img_channels, img_height, img_width = image.shape
    tgt_height, tgt_width = target_shape
    
    # Create padded arrays
    padded_image = np.zeros((img_channels, tgt_height, tgt_width), dtype=image.dtype)
    padded_mask = np.zeros((tgt_height, tgt_width), dtype=mask.dtype)
    
    # Determine padding offsets
    height_offset = max((tgt_height - img_height) // 2, 0)
    width_offset = max((tgt_width - img_width) // 2, 0)
    
    # Calculate the actual size to copy
    copy_height = min(img_height, tgt_height)
    copy_width = min(img_width, tgt_width)
    
    # Copy image and mask to padded arrays
    padded_image[:, height_offset:height_offset+copy_height, width_offset:width_offset+copy_width] = image[:, :copy_height, :copy_width]
    padded_mask[height_offset:height_offset+copy_height, width_offset:width_offset+copy_width] = mask[:copy_height, :copy_width]
    
    return padded_image, padded_mask

# Function to extract patches from images and masks
def extract_patches(image, mask, patch_size=(256, 256)):
    img_patches = []
    mask_patches = []
    img_height, img_width = image.shape[1], image.shape[2]

    for i in range(0, img_height, patch_size[0]):
        for j in range(0, img_width, patch_size[1]):
            img_patch = image[:, i:i + patch_size[0], j:j + patch_size[1]]
            mask_patch = mask[i:i + patch_size[0], j:j + patch_size[1]]
            
            # Skip patches that are smaller than the desired size
            if img_patch.shape[1] != patch_size[0] or img_patch.shape[2] != patch_size[1]:
                continue
            
            img_patches.append(img_patch)
            mask_patches.append(mask_patch)
    
    return np.array(img_patches), np.array(mask_patches)

# Create dataset of image-mask patches
def get_image_mask_patches(file_pairs, patch_size=(256, 256)):
    all_img_patches = []
    all_mask_patches = []

    for image_path, shapefile_path in file_pairs.items():
        image, mask = create_mask(image_path, shapefile_path)
        image, mask = pad_to_shape(image, mask, patch_size)  # Pad images and masks to a fixed size
        img_patches, mask_patches = extract_patches(image, mask, patch_size)
        all_img_patches.append(img_patches)
        all_mask_patches.append(mask_patches)

    all_img_patches = np.vstack(all_img_patches)
    all_mask_patches = np.vstack(all_mask_patches)
    return all_img_patches, all_mask_patches

patch_size = (256, 256)  # Define the patch size
images, masks = get_image_mask_patches(file_pairs, patch_size)

# Save images and masks patches as numpy arrays
np.save('images_patches.npy', images)
np.save('masks_patches.npy', masks)

print("Image patches and mask patches saved as numpy arrays.")

Image patches and mask patches saved as numpy arrays.


In [9]:
# Assume images_patches.npy and masks_patches.npy are already created
images = np.load('images_patches.npy')
masks = np.load('masks_patches.npy')

# Normalize images
images = images / 255.0

# Validate and correct masks
masks = validate_and_correct_masks(masks, num_classes=6)

# Convert masks to one-hot encoding
num_classes = 6  # Number of classes
masks_one_hot = tf.keras.utils.to_categorical(masks, num_classes=num_classes)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, masks_one_hot, test_size=0.2, random_state=42)


Unique values in masks before correction: [0 1 2 3 4 5 6]
Unique values in masks after correction: [0 1 2 3 4 5]


In [ ]:
# U-Net Model Definition
def unet_model(input_size=(256, 256, 8), num_classes=6):
    inputs = Input(input_size)
    
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)  # Number of output channels to match the number of classes

    model = Model(inputs=[inputs], outputs=[outputs])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the model
model = unet_model()

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Early Stopping and Model Checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_unet_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=4),
    validation_data=(X_val, y_val),
    steps_per_epoch=len(X_train) // 4,
    epochs=10,
    callbacks=[early_stopping, checkpoint]
)

# Save the final model
model.save('final_unet_model.h5')

print("Model training complete and saved.")


In [1]:
import os
import glob
import re
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.features import geometry_mask
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import glob
import re
import rasterio
import geopandas as gpd
from rasterio.features import geometry_mask

In [2]:
# Define the directories containing images and shapefiles
image_dir = "C:\\Users\\Student\\Documents\\101_walrus_data\\train\\clipped_images"
shapefile_dir = "C:\\Users\\Student\\Documents\\101_walrus_data\\train\\clipped_train"

# Use glob to list all .tif and .shp files in the specified directories
image_paths = glob.glob(f"{image_dir}/*.tif")
shapefile_paths = glob.glob(f"{shapefile_dir}/*.shp")

# Function to extract island name and date from file names
def extract_island_and_date(filename):
    pattern = r'([a-zA-Z]+)_(\d{8})'
    match = re.search(pattern, filename)
    if match:
        return match.group(1), match.group(2)
    return None, None

# Pair image and shapefile paths based on filenames
def pair_files(image_paths, shapefile_paths):
    file_pairs = {}
    for image_path in image_paths:
        island, date = extract_island_and_date(os.path.basename(image_path))
        if island is None or date is None:
            print(f"Failed to extract island and date from {image_path}")
            continue

        matching_shapefiles = [shapefile for shapefile in shapefile_paths if island in shapefile and date in shapefile]
        if not matching_shapefiles:
            print(f"No matching shapefile found for {image_path}")
            continue

        shapefile_path = matching_shapefiles[0]
        file_pairs[image_path] = shapefile_path
    return file_pairs

file_pairs = pair_files(image_paths, shapefile_paths)

# Function to create mask from shapefiles
def create_mask(image_path, shapefile_path):
    with rasterio.open(image_path) as src:
        image = src.read()  # Read all bands
        transform = src.transform

    gdf = gpd.read_file(shapefile_path)
    mask = np.zeros((src.height, src.width), dtype=np.uint8)

    # Apply label mapping
    label_mapping = {
        'ocean': 1,
        'rock': 2,
        'freshwater': 3,
        'ice': 4,
        'walrus_pure': 5,
        'walrus_mixed': 6
    }

    for _, row in gdf.iterrows():
        geom_mask = geometry_mask([row['geometry']], transform=transform, invert=True, out_shape=(src.height, src.width))
        label = label_mapping.get(row['label'], 0)  # Use 0 for any label not in the mapping
        mask[geom_mask] = label

    return image, mask

# Function to ensure masks have valid labels
def validate_and_correct_masks(masks, num_classes):
    unique_values = np.unique(masks)
    print("Unique values in masks before correction:", unique_values)
    
    # Ensure all values are within the valid range
    masks[masks >= num_classes] = 0

    unique_values = np.unique(masks)
    print("Unique values in masks after correction:", unique_values)
    return masks

# Create dataset of image-mask pairs
def get_image_mask_pairs(file_pairs):
    images = []
    masks = []

    for image_path, shapefile_path in file_pairs.items():
        image, mask = create_mask(image_path, shapefile_path)
        images.append(image)
        masks.append(mask)

    images = np.stack(images)
    masks = np.stack(masks)
    
    # Validate and correct masks
    masks = validate_and_correct_masks(masks, num_classes=6)
    
    return images, masks

No matching shapefile found for C:\Users\Student\Documents\101_walrus_data\train\clipped_images\pool_20230803_image.tif


In [3]:
# Function to pad images and masks to a fixed size
def pad_to_shape(image, mask, target_shape=(256, 256)):
    img_channels, img_height, img_width = image.shape
    tgt_height, tgt_width = target_shape
    
    # Create padded arrays
    padded_image = np.zeros((img_channels, tgt_height, tgt_width), dtype=image.dtype)
    padded_mask = np.zeros((tgt_height, tgt_width), dtype=mask.dtype)
    
    # Determine padding offsets
    height_offset = max((tgt_height - img_height) // 2, 0)
    width_offset = max((tgt_width - img_width) // 2, 0)
    
    # Calculate the actual size to copy
    copy_height = min(img_height, tgt_height)
    copy_width = min(img_width, tgt_width)
    
    # Copy image and mask to padded arrays
    padded_image[:, height_offset:height_offset+copy_height, width_offset:width_offset+copy_width] = image[:, :copy_height, :copy_width]
    padded_mask[height_offset:height_offset+copy_height, width_offset:width_offset+copy_width] = mask[:copy_height, :copy_width]
    
    return padded_image, padded_mask

# Function to extract patches from images and masks
def extract_patches(image, mask, patch_size=(256, 256)):
    img_patches = []
    mask_patches = []
    img_height, img_width = image.shape[1], image.shape[2]

    for i in range(0, img_height, patch_size[0]):
        for j in range(0, img_width, patch_size[1]):
            img_patch = image[:, i:i + patch_size[0], j:j + patch_size[1]]
            mask_patch = mask[i:i + patch_size[0], j:j + patch_size[1]]
            
            # Skip patches that are smaller than the desired size
            if img_patch.shape[1] != patch_size[0] or img_patch.shape[2] != patch_size[1]:
                continue
            
            img_patches.append(img_patch)
            mask_patches.append(mask_patch)
    
    return np.array(img_patches), np.array(mask_patches)

# Create dataset of image-mask patches
def get_image_mask_patches(file_pairs, patch_size=(256, 256)):
    all_img_patches = []
    all_mask_patches = []

    for image_path, shapefile_path in file_pairs.items():
        image, mask = create_mask(image_path, shapefile_path)
        image, mask = pad_to_shape(image, mask, patch_size)  # Pad images and masks to a fixed size
        img_patches, mask_patches = extract_patches(image, mask, patch_size)
        all_img_patches.append(img_patches)
        all_mask_patches.append(mask_patches)

    all_img_patches = np.vstack(all_img_patches)
    all_mask_patches = np.vstack(all_mask_patches)
    return all_img_patches, all_mask_patches

patch_size = (256, 256)  # Define the patch size
images, masks = get_image_mask_patches(file_pairs, patch_size)

# Save images and masks patches as numpy arrays
np.save('images_patches.npy', images)
np.save('masks_patches.npy', masks)

print("Image patches and mask patches saved as numpy arrays.")


Image patches and mask patches saved as numpy arrays.


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Assume images_patches.npy and masks_patches.npy are already created
images = np.load('images_patches.npy')
masks = np.load('masks_patches.npy')

# Normalize images
images = images / 255.0

# Convert masks to one-hot encoding
num_classes = 6  # Number of classes
masks_one_hot = tf.keras.utils.to_categorical(masks, num_classes=num_classes)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, masks_one_hot, test_size=0.2, random_state=42)

IndexError: index 6 is out of bounds for axis 1 with size 6

In [ ]:
# U-Net Model Definition
def unet_model(input_size=(256, 256, 8), num_classes=6):
    inputs = Input(input_size)
    
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)  # Number of output channels to match the number of classes

    model = Model(inputs=[inputs], outputs=[outputs])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the model
model = unet_model()

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Early Stopping and Model Checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_unet_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=4),
    validation_data=(X_val, y_val),
    steps_per_epoch=len(X_train) // 4,
    epochs=50,
    callbacks=[early_stopping, checkpoint]
)

# Save the final model
model.save('final_unet_model.h5')

print("Model training complete and saved.")

In [15]:
import numpy as np

# Load the masks array
masks = np.load('masks_patches.npy')

# Check unique values in masks
unique_values = np.unique(masks)
print("Unique values in masks:", unique_values)


Unique values in masks: [0 1 2 3 4 5 6]


In [16]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load the data
images = np.load('images_patches.npy')
masks = np.load('masks_patches.npy')

# Check and print unique values in masks
unique_values = np.unique(masks)
print("Unique values in masks before correction:", unique_values)

# Ensure mask values are within the range [0, 5]
masks = np.clip(masks, 0, 5)

# Check and print unique values in masks again
unique_values = np.unique(masks)
print("Unique values in masks after correction:", unique_values)

# Normalize images
images = images / 255.0

# Convert masks to one-hot encoding
num_classes = 6  # Number of classes
masks_one_hot = tf.keras.utils.to_categorical(masks, num_classes=num_classes)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, masks_one_hot, test_size=0.2, random_state=42)


Unique values in masks before correction: [0 1 2 3 4 5 6]
Unique values in masks after correction: [0 1 2 3 4 5]
